In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Тестовое задание - Core ML

In [2]:
rating = pd.read_csv('../input/movielens-20m-dataset/rating.csv')  # загружаем данные
movie = pd.read_csv('../input/movielens-20m-dataset/movie.csv')
tag = pd.read_csv('../input/movielens-20m-dataset/tag.csv')

## Первый взгляд на датасет

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

По гистограмме можно проследить, что бОльшая часть фильмов имеет до 10 000 оценок - посмотрим подробнее.

In [4]:
y1 = rating[['movieId','userId']].groupby(by = 'movieId').count()['userId']

fig, ax = plt.subplots()

plt.hist(y1,
        color = 'pink',
        bins = 8)
plt.title('Количество оценок для каждого фильма')
plt.xlabel('количество оценок')


fig.set_figwidth(14)    
fig.set_figheight(8)

In [5]:
movie_count = rating[['movieId','userId']].groupby(by = 'movieId').count().sort_values(by = 'userId', ascending = False)
movie_count.userId.quantile([0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.75,0.85])

Половина фильмов имеет менее 18 оценок от зрителей, в то же время количество отметок при приближении к максимуму не растёт неестественно - можно полагать, что данный агрегатор фильмов эффективно противостоит "накрутке"

In [6]:
movie_count_df = pd.DataFrame({'movieId':movie_count.index.tolist(), 'movie_count':list(movie_count['userId'])}) # таблица с id фильмов и числом их отметок
rating_count = pd.merge(rating, movie_count_df, how='left',
         left_on = 'movieId', right_on = 'movieId') # добавляем в rating столбец с количеством отметок для каждого фильма

In [7]:
y2 = rating[['movieId','userId']].groupby(by = 'userId').count()['movieId']

fig, ax = plt.subplots()

plt.hist(y2,
        color = 'grey',
        bins = 8)
plt.title('Количество оценок для каждого пользователя')
plt.xlabel('количество оценок')


fig.set_figwidth(14)    
fig.set_figheight(8)

In [8]:
user_count = rating[['movieId','userId']].groupby(by = 'userId').count().sort_values(by = 'movieId', ascending = False)
user_count.movieId.quantile([0.1,0.25,0.5,0.75,0.85,0.95,0.97])

In [10]:
user_count_df = pd.DataFrame({'userId':user_count.index.tolist(), 'user_count':list(user_count['movieId'])}) # таблица с id фильмов и числом их отметок
rating_count = pd.merge(rating_count, user_count_df, how='left',
         left_on = 'userId', right_on = 'userId') # добавляем в rating столбец с количеством отметок для каждого фильма

# Предобработка данных

In [11]:
# удаляем из исходного датасета фильмы с небольшим количеством отметок, а так же пользователей с небольшим или аномально большим (кватиль 95%) количеством отметок

rating_count = rating_count[((rating_count['movie_count']>=50) & (rating_count['user_count']>=25) & (rating_count['user_count']<=520))]

По причине нехватки оперативной памяти и ограничении на объём входных данных некоторых функий pandas извлечём из датасета выборку. Это может привести к меньшей репрезентативности,но пайплайн решения существенно не изменится.

In [12]:
rating_sample = rating_count.sample(frac = 0.001)

In [13]:
from sklearn.model_selection import train_test_split 

# расщепим на тестовую и обучающую выборку
X_train, X_test, y_train, y_test = train_test_split(rating_sample[['userId','movieId']].values, rating_sample['rating'], test_size=0.3, # доля трейна
                                                    random_state = 88) # установим зерно

In [14]:
train_df = pd.DataFrame({'userId':X_train[:,0], 'movieId':X_train[:,1], 'rating':y_train.tolist()})
train_df

После расщепления на трейн и тест некоторые фильмы из трейна могут не оказаться в тесте и наоборот - исправим это.

In [15]:
test_df = pd.DataFrame({'userId':X_test[:,0], 'movieId':X_test[:,1], 'rating':y_test.tolist()})

movie_list = list(train_df['movieId'].unique())
movie_list_test = list(test_df['movieId'].unique())

for i in range(len(movie_list)):
    if movie_list[i] not in movie_list_test:
        test_df = test_df.append(pd.DataFrame({'userId':[66625], 'movieId':[movie_list[i]],'rating':[0]}))
        
for i in range(len(movie_list_test)):
    if movie_list_test[i] not in movie_list:
        train_df = train_df.append(pd.DataFrame({'userId':[15617], 'movieId':[movie_list_test[i]],'rating':[0]}))
    
test_df

In [19]:
movie_matrix_train = train_df.pivot(index='userId', columns='movieId', values='rating').fillna(0).sort_values(by = 'userId')
movie_matrix_train.head()

# SVD - разложение

In [23]:
# U - 
U, S, Vh = np.linalg.svd(movie_matrix_train)

In [24]:
U.shape

In [31]:
Vh.shape

In [53]:
S.shape

U - матрица с факторами пользователей(по строкам), Vh - матрица с факторами фильмов(по столбцам). Для прогноза оценки данного пользователя для данного фильма нужно скалярно перемножить значимые (первые 3212) факторов пользователя на факторы данного фильма.

In [85]:
indexes = movie_matrix_train.index.tolist()